# Reto Ejercicio

In [9]:
import pandas as pd
import numpy as np
import math
import random

Una empresa desea elaborar un plan de repartición entre sus diversos clientes. La empresa desea reducir el número de unidades de transporte a utilizar y el tiempo de recorrido. Cada unidad de transporte tiene una capacidad de 8 m^3.  El número de productos que compra un cliente esta dado por una distribución de poisson con λ=2.  La empresa solo ofrece 5 productos a la venta cuyo volumen se muestra en la siguiente tabla.

In [179]:
#Datos iniciales

l = 2
cap_camion = 8 #Capacidad de 8 m^3 del camion
productos_volumen = pd.DataFrame({"Producto": ["P1","P2","P3","P4","P5"] ,"Volumen (m^3)" :[1,0.8,1.2,1.5,0.6]})
venta_dia = pd.DataFrame({"Producto": ["P1","P2","P3","P4","P5"] ,"Venta promedio/dia" :[119,140,155,158,176]}) #Venta promedio al dia

In [180]:
productos_volumen

,Producto,Volumen (m^3)
0,P1,1.0
1,P2,0.8
2,P3,1.2
3,P4,1.5
4,P5,0.6


Por políticas de la empresa solo se puede vender a lo más tres productos a cada cliente. En un estudio realizado se encuentra que la venta promedio de cada uno de los productos en un día son los siguientes:

In [12]:
venta_dia

,Producto,Venta promedio/dia
0,P1,119
1,P2,140
2,P3,155
3,P4,158
4,P5,176


### Primera parte

Simule la venta de la empresa para un día en base a los datos anteriores, para ello considere solo 10 
clientes.  Para ello

*	Determine el número de productos que comprara cada cliente. (Distribución de Poisson)


In [13]:
prob = pd.DataFrame({"X": [0,1,2,3]})

# Set the "X" column as the index
prob.set_index("X", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
prob.reset_index(drop=True, inplace=True)

prob["p(x)"] = [ ((l ** x) *  math.exp(-l)) / math.factorial(x) for x in range(0,4)] 
prob["p(x)"] = prob["p(x)"] / prob["p(x)"].sum()  #normalizamos al dividir entre la suma total de todas las probabilidades

#Probabilidad acumulada
acumul = []
for i in range(0,prob.shape[0]):
    value = prob.loc[0:i, "p(x)"].sum() 
    acumul.append(value)

prob["p(x) acumulada"] = acumul 

#Limite inferior y superior

prob["Lim inferior"] =  [prob["p(x) acumulada"][i] - prob["p(x)"][i]  for i in range(prob.shape[0])]
prob["Lim superior"] =  [prob["p(x) acumulada"][i]  for i in range(prob.shape[0])]


In [14]:
prob

,p(x),p(x) acumulada,Lim inferior,Lim superior
0,0.157895,0.157895,0.000000,0.157895
1,0.315789,0.473684,0.157895,0.473684
2,0.315789,0.789474,0.473684,0.789474
3,0.210526,1.000000,0.789474,1.000000


In [25]:
num_pedidos = pd.DataFrame({
    "Cliente": [i for i in range(10)]

})

# Set the "X" column as the index
num_pedidos.set_index("Cliente", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
num_pedidos.reset_index(drop=True, inplace=True)

#Numero de num_pedidos que pedira cada cliente en base a las probabilidades
numPedidos = []
num_aleatorio = []
for i in range(num_pedidos.shape[0]):
    p = random.uniform(0,1)
    num_aleatorio.append(p)
    for j in range(prob.shape[0]):
        if (prob["Lim inferior"][j] <= p and p < prob["Lim superior"][j]):
            numPedidos.append(j) #Numero de num_pedidos
        else:
            continue

num_pedidos["Aleatorio"] = num_aleatorio
num_pedidos["Numero de pedidos"] = numPedidos

In [26]:
num_pedidos

,Aleatorio,Numero de pedidos
0,0.561470,2
1,0.458427,1
2,0.584184,2
3,0.986629,3
4,0.420115,1
5,0.525192,2
6,0.869530,3
7,0.450017,1
8,0.525651,2
9,0.825736,3


* Determine los productos a comprar (Probabilidad relativa)

In [38]:
venta_dia["Prob Relativa"] = [venta_dia["Venta promedio/dia"][x] / venta_dia["Venta promedio/dia"].sum() for x in range(venta_dia.shape[0])]

#Probabilidad acumulada
acumul = []
for i in range(0,venta_dia.shape[0]):
    value = venta_dia.loc[0:i, "Prob Relativa"].sum() 
    acumul.append(value)

venta_dia["Prob acumulada"] = acumul 

#Limite inferior y superior

venta_dia["Lim inferior"] =  [venta_dia["Prob acumulada"][i] - venta_dia["Prob Relativa"][i]  for i in range(venta_dia.shape[0])]
venta_dia["Lim superior"] =  [venta_dia["Prob acumulada"][i]  for i in range(venta_dia.shape[0])]


In [39]:
venta_dia

,Producto,Venta promedio/dia,Prob Relativa,Prob acumulada,Lim inferior,Lim superior
0,P1,119,0.159091,0.159091,0.000000,0.159091
1,P2,140,0.187166,0.346257,0.159091,0.346257
2,P3,155,0.207219,0.553476,0.346257,0.553476
3,P4,158,0.211230,0.764706,0.553476,0.764706
4,P5,176,0.235294,1.000000,0.764706,1.000000


In [111]:
tipo_pedidos = pd.DataFrame({
    "Cliente": [i for i in range(10)]

})

# Set the "X" column as the index
tipo_pedidos.set_index("Cliente", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
tipo_pedidos.reset_index(drop=True, inplace=True)

def tipoPedido(num_pedidos,prob_tipoPedido):
    """    
    tipo_pedido = {
        "P1": 0,
        "P2": 0,
        "P3": 0,
        "P4": 0,
        "P5": 0
    }
    """

    tipo_pedido = [0,0,0,0,0]

    for i in range(num_pedidos):

        prob_pedido = random.uniform(0,1)
        #print(prob_pedido)

        for j in range(prob_tipoPedido.shape[0]):           
            if (prob_tipoPedido["Lim inferior"][j] <= prob_pedido and prob_pedido < prob_tipoPedido["Lim superior"][j]):
                tipo_pedido[j] += 1
            else:
                continue
    
    return tipo_pedido
    

In [116]:
tipoPedido(3,venta_dia)

[1, 0, 0, 1, 1]

In [161]:
def designarPedidos(num_pedidos,prob_tipoPedido):

    numPedidos = num_pedidos["Numero de pedidos"] #lista de numeros de pedidos por cada cliente

    pedidos = {
        "P1": [],
        "P2": [],
        "P3": [],
        "P4": [],
        "P5": []
    }

    for num in numPedidos:
        listaPedidos = tipoPedido(num,prob_tipoPedido)
        index = 0
        values = list(pedidos.values())
        for key in pedidos.keys():
            values[index].append(listaPedidos[index]) #numero de pedidos por tipo de pedido
            index += 1
        
        for index,key in enumerate(pedidos):
            pedidos[key] = values[index]
    
    
    pedidosFinal = pd.concat([num_pedidos, pd.DataFrame(pedidos)], axis=1)
    return pedidosFinal

                
    

In [166]:
productos_porCliente= designarPedidos(num_pedidos,venta_dia)
productos_porCliente

,Aleatorio,Numero de pedidos,P1,P2,P3,P4,P5
0,0.561470,2,0,1,0,1,0
1,0.458427,1,0,0,0,0,1
2,0.584184,2,0,0,2,0,0
3,0.986629,3,1,0,1,1,0
4,0.420115,1,0,0,0,0,1
5,0.525192,2,0,0,1,1,0
6,0.869530,3,0,0,3,0,0
7,0.450017,1,0,1,0,0,0
8,0.525651,2,0,0,1,0,1
9,0.825736,3,2,0,0,1,0


In [181]:
productos_volumen

,Producto,Volumen (m^3)
0,P1,1.0
1,P2,0.8
2,P3,1.2
3,P4,1.5
4,P5,0.6


In [191]:
def calcularVolumen(df,volumenes):

    volumenTotalPorPedidos = []

    for idx,productos in df.iterrows():
        volumenTotal = 0
        for i in range(len(volumenes)):
             volumenTotal += productos[i] * volumenes[i]
        volumenTotalPorPedidos.append(volumenTotal)

    return volumenTotalPorPedidos    
        


In [193]:
productos_porCliente["Volumen por pedidos (m^3)"]= calcularVolumen(productos_porCliente.iloc[:,2:productos_porCliente.shape[1]],list(productos_volumen["Volumen (m^3)"]))

In [196]:
productos_porCliente

,Aleatorio,Numero de pedidos,P1,P2,P3,P4,P5,Volumen por pedidos (m^3)
0,0.561470,2,0,1,0,1,0,2.3
1,0.458427,1,0,0,0,0,1,0.6
2,0.584184,2,0,0,2,0,0,2.4
3,0.986629,3,1,0,1,1,0,3.7
4,0.420115,1,0,0,0,0,1,0.6
5,0.525192,2,0,0,1,1,0,2.7
6,0.869530,3,0,0,3,0,0,3.6
7,0.450017,1,0,1,0,0,0,0.8
8,0.525651,2,0,0,1,0,1,1.8
9,0.825736,3,2,0,0,1,0,3.5


### Algoritmo de creacion de rutas